In [47]:
import os
import re
import requests
import statistics
from src.najdi_rok import najdi_rok
import pandas as pd

In [48]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [49]:
kam = 'data_raw/nobelistvo'

In [50]:
if not os.path.exists(kam):
    os.makedirs(kam)

In [51]:
koho = pd.read_json(os.path.join('data_raw','wikidata_nobelovky_pro_filtr.json'))

In [52]:
sloupce = ['100','245','700']

In [53]:
df = pd.read_parquet(os.path.join('data/cnb_sloupce','008.parquet'))
for s in sloupce:
    df = df.merge(pd.read_parquet(os.path.join('data/cnb_sloupce',f'{s}.parquet')), left_index=True, right_index=True, how='outer')

In [54]:
df.sample(10)

,008,100_ind1,100_a,100_7,100_4,100_d,100_q,100_c,100_b,100_e,245_ind1,245_ind2,245_a,245_b,245_c,245_n,245_p,245_h,245_f,245_s,245_v,700_ind1,700_a,700_4,700_d,700_7,700_t,700_q,700_l,700_ind2,700_c,700_b,700_i,700_n,700_k,700_m,700_r,700_p,700_o,700_s,700_e,700_6,700_x,700_f,700_g
001,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
nkc20213344479,[210726s2020 xr ab f 000 0 cze],[1],"[Tichá Bambasová, Sylva]",[ola20211123259],[aut],None,None,None,None,None,[1],[0],"[Křenovice - ""Hradisko"" a jeho zázemí v době h...",None,[Sylva Tichá Bambasová],None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bk196001339,[980527s1960 xr cze],[1],"[Závada, Vilém,]",[jk01152210],[aut],[1905-1982],None,None,None,None,[1],[0],[Na zemi /],None,[[Autor:] Vilém Závada],None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nkc20061649485,[070419s2006 xr a b 000 f cze],[1],"[Casalis, Anna]",[mzk2005309248],[aut],None,None,None,None,None,[1],[0],[Myšák Eda jede k babičce a dědovi /],None,[[texty Anna Casalis] ; ilustrace Marco Campan...,None,None,None,None,None,None,[1],"[Campanella, Marco]",[ill],None,[mzk2003187366],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
cpg20213344843,[210728s2021 xr nnn g nncze],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[1],[0],[Hory 2022 =],[Mountains 2022 /],[foto Getty Images],None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cpz20193135803,[190927t20192019xr nnnng z n cze],[1],"[Negroni, Christine,]",[mzk20181014514],[aut],[1956-],None,None,None,None,[1],[0],[Letecké katastrofy /],None,[Christine Negroni ; z anglického originálu Th...,None,None,None,None,None,None,"[1, 1, 1]","[Sedláková, Hana, Veldová, Apolena,, Preiss, M...","[trl, nrt, prf]","[None, 1964-, 1973-]","[None, xx0119239, jn20000402050]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
cpg20152681599,[150303s2012 xr nnn g kncze],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[1],[0],"[Hraj kapelo, hraj 2012]",[16. mezinárodní festival české a moravské dec...,None,None,None,[[grafika] :],None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nkc20051626502,[051011s2005 xr a f f 000 0 cze],[1],"[Rasocha, Vlastimil,]",[mzk2003171455],[aut],[-2018],None,None,None,None,[1],[0],[Ochrana brambor proti mandelince bramborové /],None,[Vlastimil Rasocha],None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cpk19970188732,[970325s1997 xr e 000 p cze],[1],"[Zahradníček, Jan,]",[jk01151940],[aut],[1905-1960],None,None,None,None,[1],[0],[Skrytý pramen :],[výbor z veršů /],[Jan Zahradníček],None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cpk19980369216,[980820s1998 xr ae e f 000 0 cze],[1],"[Etzold, Hans-Rüdiger,]",[xx0020263],[aut],[1940-],None,None,None,None,[1],[0],[Údržba a opravy automobilů Ford Fiesta :],[Courier/Classic benzin : Courier/Classic dies...,[Hans-Rüdiger Etzold ; [přeloženo z německého ...,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
for index, row in koho.iterrows():
    nazev_souboru = f"nobelovka_{row['nobelovka']}_{row['100_7']}.parquet"
    if nazev_souboru not in os.listdir(kam):
        print(row['nobelovka'])
        df_slice = df[df['100_7'].astype(str).str.contains(row['100_7'], na=False) | df['700_7'].astype(str).str.contains(row['100_7'], na=False)]
        df_slice = df_slice.explode('008')
        df_slice['rok'] = df_slice['008'].apply(lambda x: najdi_rok(x))
        df_slice['nobelovka'] = row['nobelovka']
        df_slice['titul'] = df_slice['245_a'].apply(lambda x: x[0].replace(' /','').replace(' :','').replace(';','').strip() if x != None else None)
        df_slice.to_parquet(os.path.join(kam, nazev_souboru))

1988
1994
